# How to train the dataset?

In [1]:
# Lets first install hub
from IPython.display import clear_output
!pip install hub
clear_output()

In [4]:
# Import necessary packages
import hub
import torch
from torchvision import transforms
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
from matplotlib import pyplot as plt

In [3]:
ds_train = hub.load('/kaggle/input/food-recognition-2022/hub/train')
ds_val = hub.load('/kaggle/input/food-recognition-2022/hub/val')
ds_test = hub.load('/kaggle/input/food-recognition-2022/hub/test')

In [5]:
global class_labels;
class_labels = ds_train.categories.info.class_names

In [29]:
def transform(data):
    image, bboxes = data['images'], data['boxes']
    masks, categories = data['masks'], data['categories']
    
    transform_img_mask = A.PadIfNeeded(512, 512, p=1)
    image, masks = transform_img_mask(image=image, mask=masks)

    transform_all = A.Compose(
        [
            A.CenterCrop(512, 512),
            ToTensorV2(),
        ],
        bbox_params={
            'format':'coco', 
            'label_fields': ['class_labels']
        }
    )

    transformed = transform_all(
        image=image,
        mask=masks,
        bboxes=bboxes,
        class_labels=categories,
    )

    image = transformed['image']
    boxes = transformed['bboxes']
    masks = transformed['mask']
    categories = transformed['class_labels']
    
    return image, bboxes, masks, categories;

def collate_fn(batch):    
    return {
        "images" : torch.stack([x[0] for x in batch]), 
        "boxes" : [torch.tensor(x[1]) for x in batch],
        "masks" : [x[2] for x in batch],
        "categories" : [torch.tensor(x[3]) for x in batch]
    }

In [30]:
dataloader = ds_train.pytorch(num_workers = 0,
                        batch_size = 1,
                        transform = transform,
                        tensors = ['images', 'boxes', 'masks', 'categories'],
                        collate_fn = collate_fn,) 

for batch in dataloader:
    break;

error: OpenCV(4.5.4) :-1: error: (-5:Bad argument) in function 'copyMakeBorder'
> Overload resolution failed:
>  - src data type = 0 is not supported
>  - Expected Ptr<cv::UMat> for argument 'src'


In [19]:
batch['images']

tensor([[[[ 70,  58,  57,  ..., 209, 214, 215],
          [ 67,  61,  58,  ..., 210, 213, 214],
          [ 73,  65,  57,  ..., 210, 212, 213],
          ...,
          [ 79,  81,  78,  ..., 105, 115, 127],
          [ 81,  83,  88,  ...,  97,  92, 108],
          [ 87,  87,  89,  ...,  92,  84,  97]],

         [[ 49,  37,  36,  ...,  99, 104, 105],
          [ 47,  40,  37,  ..., 100, 103, 104],
          [ 53,  45,  36,  ..., 100, 102, 103],
          ...,
          [ 28,  30,  27,  ...,  81,  91, 103],
          [ 19,  21,  26,  ...,  73,  68,  84],
          [ 15,  15,  20,  ...,  68,  60,  73]],

         [[ 58,  46,  43,  ..., 144, 149, 150],
          [ 56,  49,  46,  ..., 145, 148, 149],
          [ 62,  54,  45,  ..., 145, 147, 148],
          ...,
          [ 45,  47,  44,  ...,  79,  89, 101],
          [ 40,  42,  47,  ...,  71,  66,  82],
          [ 37,  37,  41,  ...,  66,  58,  71]]]], dtype=torch.uint8)